In [72]:
import pandas as pd
import random
import requests
import time

In [28]:
nomenclator = requests.get("https://resultadosprecpresidente1v.registraduria.gov.co/assets/nomenclator.json").json()

In [42]:
df_location = pd.DataFrame(nomenclator["ambitos"]["1"])
df_location

,i,n,c,s,l,m,p,r,h
0,0,COLOMBIA,00,COLOMBIA,1,0,[],[0],"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
1,1,AMAZONAS,6000,AMAZONAS,2,0,[0],[1],"[364, 540, 559, 578, 588, 650, 801, 802, 819, ..."
2,2,ANTIOQUIA,0100,ANTIOQUIA,2,0,[0],[2],"[35, 37, 57, 68, 69, 74, 75, 76, 78, 81, 82, 8..."
3,3,ARAUCA,4000,ARAUCA,2,0,[0],[3],"[91, 92, 313, 413, 824, 1013, 1079]"
4,4,ATLANTICO,0300,ATLANTICO,2,0,[0],[4],"[123, 134, 205, 210, 431, 529, 602, 615, 618, ..."
...,...,...,...,...,...,...,...,...,...
18501,18501,ZUNGO EMBARCADERO,0100080990070,ZUNGO-EMBARCADERO,6,8,[3274],[18501],[]
18502,18502,ZURICH,8800760050006,ZURICH,6,2,[2742],[18502],[]
18503,18503,11 DE NOVIEMBRE,1600001121231,11-DE-NOVIEMBRE,6,1,"[2868, 5195]",[18503],[]
18504,18504,12 DE OCTUBRE,1600001121211,12-DE-OCTUBRE,6,11,"[2868, 5195]",[18504],[]


In [43]:
len("1600001040428")

13

In [44]:
len("1600001010122000001")

19

In [57]:
df_puestos = df_location[df_location["c"].apply(len) == 13]
df_puestos

,i,n,c,s,l,m,p,r,h
5245,5245,ABRAHAM LINCOLN,1600001060603,ABRAHAM-LINCOLN,6,13,"[2758, 5189]",[5245],[]
5246,5246,ABRE EL OJO,2800048990090,ABRE-EL-OJO,6,1,[3986],[5246],[]
5247,5247,ABROJAL,1300014990001,ABROJAL,6,3,[3595],[5247],[]
5248,5248,ABU DHABI - CONSULADO,8800350050002,ABU-DHABI---CONSULADO,6,4,[2706],[5248],[]
5249,5249,ABUJA,8800115050004,ABUJA,6,1,[2686],[5249],[]
...,...,...,...,...,...,...,...,...,...
18501,18501,ZUNGO EMBARCADERO,0100080990070,ZUNGO-EMBARCADERO,6,8,[3274],[18501],[]
18502,18502,ZURICH,8800760050006,ZURICH,6,2,[2742],[18502],[]
18503,18503,11 DE NOVIEMBRE,1600001121231,11-DE-NOVIEMBRE,6,1,"[2868, 5195]",[18503],[]
18504,18504,12 DE OCTUBRE,1600001121211,12-DE-OCTUBRE,6,11,"[2868, 5195]",[18504],[]


In [88]:
polling_ids = []
for i, df_tmp in df_puestos.iterrows():
    loc_id = df_tmp["c"]
    loc_name = df_tmp["n"]
    polling_places = df_tmp["m"]
    
    tmp_ids = [str(x).zfill(6) for x in range(1, polling_places + 1)]
    
    for _ in tmp_ids:
        polling_ids.append(f"{loc_id}{_}")

In [109]:
# Get data from polling place
data_output = []
data_polling = []

_round = "1v"

for polling_id in polling_ids[0:2]:
    data = requests.get(f"https://resultadosprecpresidente{_round}.registraduria.gov.co/json/ACT/PR/{polling_id}.json").json()
    tmp = pd.DataFrame([x["act"]["cantotabla"][0] for x in data["camaras"][0]["partotabla"]])
    tmp["polling_id"] = polling_id

    df_polling_place = pd.DataFrame([data["totales"]["act"]])
    df_polling_place["polling_id"] = polling_id
    
    tm_slp = random.uniform(0.05, 0.1)
    time.sleep(tm_slp)

    data_output.append(tmp)
    data_polling.append(df_polling_place)

In [112]:
df_voting = pd.concat(data_output, ignore_index=True)
df_voting.head()

,amb,codcan,nomcan,apecan,nomcan2,apecan2,vot,pvot,carg,pref,empate,polling_id
0,1600001060603000001,6,GUSTAVO,PETRO,FRANCIA,MÁRQUEZ MINA,62,"35,42%",0,1,0,1600001060603000001
1,1600001060603000001,1,RODOLFO,HERNÁNDEZ,MARELEN,CASTILLO,59,"33,71%",0,1,0,1600001060603000001
2,1600001060603000001,3,FEDERICO,GUTIÉRREZ,RODRIGO,LARA,43,"24,57%",0,1,0,1600001060603000001
3,1600001060603000001,4,SERGIO,FAJARDO,LUIS GILBERTO,MURILLO,6,"3,42%",0,1,0,1600001060603000001
4,1600001060603000001,2,JOHN MILTON,RODRÍGUEZ,SANDRA,DE LAS LAJAS TORRES,1,"0,57%",0,1,0,1600001060603000001


In [113]:
df_polling = pd.concat(data_polling, ignore_index=True)
df_polling.head()

,metota,mesesc,pmesesc,meserr,centota,carg,cargElectos,cargEmpatados,cargtota,votant,pvotant,absten,pabsten,votnul,pvotnul,votnma,pvotnma,polling_id
0,1,1,100%,0,400,0,0,0,0,176,"44,00%",224,"56,00%",1,"0,56%",0,0%,1600001060603000001
1,1,1,100%,0,400,0,0,0,0,207,"51,75%",193,"48,25%",1,"0,48%",0,0%,1600001060603000002
